In [7]:
import pandas as pd
import numpy as np

path ='/home/dragos/Projects/Longformer/data/IMDB.csv'
data = pd.read_csv(path)
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [40]:
from torchdata.datapipes.iter import IterableWrapper, FileOpener
from torch.utils.data import DataLoader, random_split

datapipe = IterableWrapper([path])
datapipe = FileOpener(datapipe, mode='b')   
datapipe = datapipe.parse_csv(skip_lines=1)

# Convert the datapipe to a list
data_list = list(datapipe)

# Split the data into train and test sets
train_ratio = 0.8
train_size = int(train_ratio * len(data_list))
test_size = len(data_list) - train_size
train_data, test_data = random_split(data_list, [train_size, test_size])

# Convert the train_data and test_data back to datapipe if needed
train_datapipe = IterableWrapper(train_data)
test_datapipe = IterableWrapper(test_data)


["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [42]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator 

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)
        
def get_vocab(data_iter):
    vocab = build_vocab_from_iterator(yield_tokens(data_iter),
                                        specials=["<UNK>", "<PAD>"],
                                        max_tokens=20000)
    vocab.set_default_index(vocab["<UNK>"])
    return vocab

vocab = get_vocab(train_datapipe)

In [45]:
print(vocab.get_itos()[:100])

['<UNK>', '<PAD>', 'the', '.', ',', 'and', 'a', 'of', 'to', "'", 'is', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'with', 'for', 'movie', 'but', 'film', ')', '(', 'you', 't', 'on', 'not', 'are', 'he', 'his', 'have', 'be', 'one', '!', 'at', 'all', 'they', 'by', 'an', 'who', 'from', 'so', 'like', 'there', 'or', 'just', 'her', 'about', 'if', 'out', 'has', '?', 'what', 'some', 'good', 'can', 'more', 'very', 'when', 'she', 'my', 'no', 'even', 'would', 'up', 'time', 'which', 'only', 'really', 'story', 'their', 'see', 'had', 'me', 'were', 'we', 'well', 'much', 'than', '-', 'been', 'bad', 'other', 'get', 'people', 'do', 'great', 'will', 'also', 'into', 'because', 'how', 'don', 'him', 'most', 'first', 'its']


In [59]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch


def text_transform(text):
    tokenized_text = torch.tensor([vocab[token] for token in tokenizer(text)])
    return tokenized_text

def label_transform(label):
    return 1 if label == 'pos' else 0

def collate_batch(batch): 
    label_list, text_list = [], [] 
    
    for (_text, _label) in batch: 
        label_list.append(label_transform(_label)) 
        processed_text = torch.tensor(text_transform(_text)) 
        text_list.append(processed_text) 

    return torch.tensor(label_list), pad_sequence(text_list) 

train_dataloader = DataLoader(train_datapipe, batch_size=8, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_datapipe, batch_size=8, shuffle=True, collate_fn=collate_batch)

i, (label, text) = next(enumerate(train_dataloader))
print(f"Label size: {label.size()}\nText size: {text.size()}")


Label size: torch.Size([8])
Text size: torch.Size([468, 8])


/home/dragos/Projects/Longformer/lib/python3.10/site-packages/torch/utils/data/graph_settings.py:103: UserWarning: `shuffle=True` was set, but the datapipe does not contain a `Shuffler`. Adding one at the end. Be aware that the default buffer size might not be sufficient for your task.
  warnings.warn(
/tmp/ipykernel_22119/4106039430.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  processed_text = torch.tensor(text_transform(_text))
